In [1]:
import os

In [2]:
GH_BASE = os.path.expanduser('~/github')
ORG = 'etcbc'
REPO = 'linksyr'
PIL_PATH = 'data/peshitta/pil'
BOOK_DATA = f'{GH_BASE}/{ORG}/{REPO}/{PIL_PATH}/books'

In [19]:
def doText(book, chapter, verse, text, results):
  results['chars'] |= set(''.join(text.strip().split()))

In [20]:
def doBooks():
  bookInfo = {}
  with os.scandir(BOOK_DATA) as bookDir:
    for bookEntry in bookDir:
      if not bookEntry.is_file():
        continue
      bookFile = bookEntry.name
      thisBookInfo = {}
      results = dict(chars=set())
      with open(f'{BOOK_DATA}/{bookFile}') as bh:
        (curChapter, curVerse, curText) = (None, None, None)
        for line in bh:
          if line.startswith('%'):
            comps = line[1:-1].split()
            if len(comps) == 0:
              continue
            keyword = comps[0]
            args = comps[1:]
            if keyword == 'bookname':
              thisBookInfo['acro'] = ' '.join(args)
            elif keyword == 'language':
              thisBookInfo['language'] = ' '.join(args)
            elif keyword == 'verse':
              if curVerse:
                doText(bookFile, curChapter, curVerse, curText, results)
              (curChapter, curVerse) = args[0].split(',')
              curText = ''
          elif curVerse:
            curText += line
      thisBookInfo['chars'] = ''.join(sorted(results['chars']))
      bookInfo[bookFile] = thisBookInfo
  return bookInfo

In [24]:
bookInfo = doBooks()

allChars = set()
for thisBookInfo in bookInfo.values():
  allChars |= set(thisBookInfo['chars'])
  
sortedChars = ''.join(sorted(allChars))

print(f'ALL CHARS = {sortedChars}')

for (book, thisBookInfo) in sorted(bookInfo.items()):
  acro = thisBookInfo['acro']
  lang = thisBookInfo['language']
  chars = thisBookInfo['chars']
  print(f'{acro:<5} = {book:<20} in {lang:<7} having {chars}')

ALL CHARS = !"#*./:<=>@ABCDEGHJKLMNPQRSTVWXYZ\^_o
1Chr  = 1Chronicles          in syriac  having "#.:<=>BCDGHJKLMNPQRSTVWXYZ\^_
1Rg   = 1Kings               in syriac  having "#.<=>BCDGHJKLMNPQRSTVWXYZ^
1Mc   = 1Maccabees_A         in syriac  having "*<>BCDGHJKLMNPQRSTVWXYZ
1Mc   = 1Maccabees_B         in syriac  having "#.<=>BCDGHJKLMNPQRSTVWXYZ^
1Sm   = 1Samuel              in syriac  having "#<>BCDGHJKLMNPQRSTVWXYZ^
2Chr  = 2Chronicles          in syriac  having "#<>BCDGHJKLMNPQRSTVWXYZ^
2Rg   = 2Kings               in syriac  having "#.<=>BCDGHJKLMNPQRSTVWXYZ^
2Mc   = 2Maccabees           in syriac  having "<>BCDGHJKLMNPQRSTVWXYZ
2Sm   = 2Samuel              in syriac  having "#<>BCDGHJKLMNPQRSTVWXYZ^
3Esr  = 3Esdras              in syriac  having "#*.<=>BCDGHJKLMNPQRSTVWXYZ^
3Mc   = 3Maccabees           in syriac  having ".<=>BCDGHJKLMNPQRSTVWXYZ^
4Esr  = 4Esdras              in syriac  having !"#*.<=>BCDGHJKLMNPQRSTVWXYZ^_o
4Mc   = 4Maccabees           in syriac  having "#*.:<=>@